In [ ]:
# week 3
try:
  import colab
  !pip install --upgrade pip
except:
  pass

     |████████████████████████████████| 2.1 MB 3.6 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [ ]:
# Write a line of code to install TFX

# 1 => CODE HERE # 


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from absl import logging
import urllib.request
import tempfile
import pandas as pd
logging.set_verbosity(logging.INFO)  # Set default logging level.

import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

# you should see the following:
# TensorFlow version: 2.8.0
# TFX version: 1.7.1

TensorFlow version: 2.8.0
TFX version: 1.7.1


In [ ]:


# Define the following variables:
# PIPELINE_NAME to give a name to your pipeline
# 2 => CODE HERE # 


# PIPELINE_ROOT for output directory to store artifacts generated from the pipeline.
# 3 => CODE HERE # 


# METADATA_PATH for storing meta data
# 4 => CODE HERE # 


# SERVING_MODEL_DIR to deploy your model
# 5 => CODE HERE # 



In [ ]:


DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.

# read your CSV file. Convert column Species from categorical string to int values using a mapping. store the new file in a temporary directory. 
# _data_filepath should point to your converted CSV file
# 6 => CODE HERE # 

_data_filepath = None


In [ ]:
# take a look at the first five lines of your csv file
# 7 => CODE HERE # 


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,2
1,2,4.9,3.0,1.4,0.2,2
2,3,4.7,3.2,1.3,0.2,2
3,4,4.6,3.1,1.5,0.2,2
4,5,5.0,3.6,1.4,0.2,2
5,6,5.4,3.9,1.7,0.4,2
6,7,4.6,3.4,1.4,0.3,2
7,8,5.0,3.4,1.5,0.2,2
8,9,4.4,2.9,1.4,0.2,2


In [ ]:
#  define _trainer_module_file variable to save the name of your training file.
_trainer_module_file = 'iris_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

# define the list of features in _FEATURE_KEYS variable
# 8 => CODE HERE # 
_FEATURE_KEYS = []

# define your target variable _LABEL_KEY
# 9 => CODE HERE # 
_LABEL_KEY = ''

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  # compelete your model architecture here
  # 10 => CODE HERE # 
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

Overwriting iris_trainer.py


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

INFO:absl:Generating ephemeral wheel package for '/content/iris_trainer.py' (including modules: ['penguin_trainer', 'iris_trainer']).
INFO:absl:User module package has hash fingerprint version 195d51a39b15058ab9a05df04b5656c8010ef3c6d2d9878f4ebc1c43ce8f4361.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmptbgd8beh/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpyf4g4tng', '--dist-dir', '/tmp/tmp0j78a9sd']
INFO:absl:Successfully built user code wheel distribution at 'pipelines/iris-pipeline/_wheels/tfx_user_code_Trainer-0.0+195d51a39b15058ab9a05df04b5656c8010ef3c6d2d9878f4ebc1c43ce8f4361-py3-none-any.whl'; target user module is 'iris_trainer'.
INFO:absl:Full user module path is 'iris_trainer@pipelines/iris-pipeline/_wheels/tfx_user_code_Trainer-0.0+195d51a39b15058ab9a05df04b5656c8010ef3c6d2d9878f4ebc1c43ce8f4361-py3-none-any.whl'
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec

100/100 [==============================] - 1s 4ms/step - loss: 0.5279 - sparse_categorical_accuracy: 0.7685 - val_loss: 0.1166 - val_sparse_categorical_accuracy: 0.9800


INFO:tensorflow:Assets written to: pipelines/iris-pipeline/Trainer/model/11/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/iris-pipeline/Trainer/model/11/Format-Serving/assets
INFO:absl:Training complete. Model written to pipelines/iris-pipeline/Trainer/model/11/Format-Serving. ModelRun written to pipelines/iris-pipeline/Trainer/model_run/11
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 11 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model': [Artifact(artifact: uri: "pipelines/iris-pipeline/Trainer/model/11"
custom_properties {
  key: "name"
  value {
    string_value: "iris-pipeline:2022-05-18T01:43:48.852721:Trainer:model:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.7.1"
  }
}
, artifact_type: name: "Model"
base_type: MODEL
)], 'model_run': [Artifact(artifact: uri: "pipelines/iris-pipeline/Trainer/model_run/11"
custom_properties {
  key: "name"
  value {
    string_value: "iris-pipeline:2022-05-18T01:43:48.852721:Trainer:mo

In [ ]:
#  check serving folder to see what you have deployed


serving_model/iris-pipeline
serving_model/iris-pipeline/1652837311
serving_model/iris-pipeline/1652837311/assets
serving_model/iris-pipeline/1652837311/variables
serving_model/iris-pipeline/1652837311/variables/variables.index
serving_model/iris-pipeline/1652837311/variables/variables.data-00000-of-00001
serving_model/iris-pipeline/1652837311/keras_metadata.pb
serving_model/iris-pipeline/1652837311/saved_model.pb
serving_model/iris-pipeline/1652838192
serving_model/iris-pipeline/1652838192/assets
serving_model/iris-pipeline/1652838192/variables
serving_model/iris-pipeline/1652838192/variables/variables.index
serving_model/iris-pipeline/1652838192/variables/variables.data-00000-of-00001
serving_model/iris-pipeline/1652838192/keras_metadata.pb
serving_model/iris-pipeline/1652838192/saved_model.pb
serving_model/iris-pipeline/1652837743
serving_model/iris-pipeline/1652837743/assets
serving_model/iris-pipeline/1652837743/variables
serving_model/iris-pipeline/1652837743/variables/variables.i